In [1]:
### setup paths and load modules ###
import sys
path_to_gws = '/home/scott/Repos/GitRepos/Codes/gwsurrogate/'
sys.path.append(path_to_gws)

path_to_gwtools = '/home/scott/Repos/GitRepos/Codes/gwsurrogate/gwsurrogate/'
sys.path.append(path_to_gwtools)

from pylab import matplotlib as plt
import numpy as np
import gwsurrogate as gws
import gwtools as gwtools

### import lal and lalsimulation python packages ###
# LAL homepage: https://www.lsc-group.phys.uwm.edu/daswg/projects/lalsuite.html
import lal as lal
import lalsimulation as lalsim

In [2]:
### load an EOB surrogate ###
EOBNRv2_sur = gws.EvaluateSurrogate(path_to_gws+'tutorial/EOBNRv2_example/EOBNRv2_q1_2_NoSpin_SingleModes/')

loading surrogate mode... l2_m2


### Lesson 3: Timings of waveforms generated by EOB-LAL code and its surrogate

In [ ]:
### average time to evaluate a waveform (WARNING: code not yet optimized for speed, but should still be pretty fast...) ###
# 1000 waveforms are evaluated. This will take a few seconds...
EOBNRv2_sur.timer()

In [ ]:
### average time to evaluate a waveform (WARNING: code not yet optimized for speed, but should still be pretty fast...) ###
# 1000 waveforms are evaluated. This will take a few seconds...
EOBNRv2_sur.timer(M_eval=80.0, dist_eval=1.0, phi_ref = 0.0, f_low = 15.0)

In [ ]:
### average time to evaluate a waveform (WARNING: code not yet optimized for speed, but should still be pretty fast...) ###
# 1000 waveforms are evaluated. This will take a few seconds...
t_orig = EOBNRv2_sur.time()
t_resamp = np.linspace(t_orig.min(), t_orig.max(), num=8000)
EOBNRv2_sur.timer(M_eval=80.0, dist_eval=1.0, phi_ref = 0.0, f_low = 15.0, samples = t_resamp)

### Physical waveform evaluations (any permissible input)

In [3]:
### python interface to EOBNRv2-LAL ###
def EOBNRv2_LAL(Mtot,q,dt,fmin,Dist,inclination):
        
    M1,M2  = gwtools.Mq_to_m1m2(Mtot,q)
    hp, hc = lalsim.SimInspiralChooseTDWaveform(phiRef=0.0,deltaT=dt,m1=M1,m2=M2,s1x=0.0,s1y=0.0,s1z=0.0,\
                                                s2x=0.0,s2y=0.0,s2z=0.0,f_min=fmin,f_ref=0.0,\
                                                r=Dist,i=inclination,lambda1=0.0,lambda2=0.0,\
                                                waveFlags=None,nonGRparams=None,amplitudeO=0,phaseO=7,\
                                                approximant=lalsim.EOBNRv2)
    h      = hp.data.data + (1j)*hc.data.data
    times  = np.arange(np.size(hp.data.data))*hp.deltaT
    
    times,h = gwtools.remove_amplitude_zero(times,h)
    
    return times, h

In [4]:
### comparison for physical waveform modes ###
M    = 80.0
q    = 1.0
incl = 2.0
Dist = 1.0 * 1e6 * lal.PC_SI
Mtot = M * lal.MSUN_SI
fmin = 10.0
dt   = 1.0/2048.0

# LAL evaluation... #
time_LAL, h_LAL = EOBNRv2_LAL(Mtot,q,dt,fmin,Dist,incl)

# Multimode models

In [6]:
### Generate a set of modes for some z-rotation and time shift ###
modes_to_evaluate, t_ref, hp_modes, hc_modes = \
    EOBNRv2_sur(q=1.5,M=10.0,dist=2.0,z_rot=0.2,mode_sum=False,fake_neg_modes=True)
t_ref = t_ref + 1.0

h_sphere = gwtools.h_sphere_builder(modes_to_evaluate, hp_modes, hc_modes, t_ref)

 # h_ref is reference modes we seek to match to
hp, hc = h_sphere(t_ref)
h_ref = hp + 1.0j*hc

#gwtools.plot_pretty(t_ref,[h_ref_22.real,h_ref_22.imag],showQ=True)

ell_match = 2
m_match = 2
mode_indx = modes_to_evaluate.index((ell_match,m_match))
h_ref_22 = h_ref[:,mode_indx]

t1, hp_22, hc_22 = \
    EOBNRv2_sur(q=1.5,M=10.0,dist=2.0,mode_sum=True,ell=[ell_match],m=[m_match])
h_22 = hp_22 + 1.0j*hc_22
    
[tmp, minimized_norm], [tc, phic], [t_align,h_align,h_align] = \
    gwtools.minimize_norm_error(t1,h_22,t_ref,h_ref_22,gwtools.euclidean_norm_sqrd,method='nelder-mead')
z_rot_opt = -phic/2
    


In [7]:
[guessed_norm_1mode, minimized_norm_1mode, min_norm_sphere], [tc, z_rot_opt], [common_times,h1_align,href_eval] = \
    gwtools.minimize_norm_error_multi_mode(t_match,h_match,t_ref,h_ref,modes,(2,2),method='nelder-mead')
    
print min_norm_sphere

9.02649101871e-28


# Multimode eob surrogates from PRX (need to update data files)

In [ ]:
### download a surrogate (this will take a moment, check the terminal for progress) ##
unzip_loc = gws.surrogate_repo.get('EOBNRv2')
# ...or specify a custom download location as 
#unzip_loc = gws.surrogate_repo.get('EOBNRv2','/home/scott/Desktop/surrogate_download/')

### if everything works you should see a surrogate file or folder(s) ###
print os.listdir(unzip_loc)  

In [ ]:
### view available surrogates. Each folder l#_m# is a separate surrogate model  ###

print 'General surrogate types: ',os.listdir(unzip_loc)

# (2,2) mode for duration 12236 M, usings 40 basis and polynomial fits
print 'q = 1-10 surrogates: ',os.listdir(unzip_loc+'/EOBNRv2_q1_10_NoSpin_SingleModes') 

# variety of modes, durations and basis
print 'q = 1-2 surrogates: ',os.listdir(unzip_loc+'/EOBNRv2_q1_2_NoSpin_SingleModes') 

In [ ]:
### load your favorite surrogates ###
EOBNRv2_22_q10_12236M = \
  gws.EvaluateSingleModeSurrogate(unzip_loc+'EOBNRv2_q1_10_NoSpin_SingleModes/'+'l2_m2_len12236M_SurID40poly/')
EOBNRv2_22_q2_191839M = \
  gws.EvaluateSingleModeSurrogate(unzip_loc+'EOBNRv2_q1_2_NoSpin_SingleModes/'+'l2_m2_len191839M_SurID20poly/')
EOBNRv2_44_q2_12238M  = \
  gws.EvaluateSingleModeSurrogate(unzip_loc+'EOBNRv2_q1_2_NoSpin_SingleModes/'+'l4_m4_len12238M_SurID15poly/')

In [ ]:
### evaluate and plot ###
t, hp, hc = EOBNRv2_22_q10_12236M(q = 8.7, M = 80.0, dist = 1.0, phi_ref = 3.1415, f_low = 15.0)
EOBNRv2_22_q10_12236M.plot_pretty(t, hp, hc,1) 

t, hp, hc = EOBNRv2_22_q2_191839M(q = 1.7, M = 7.0, dist = 1.0, phi_ref = 17.0, f_low = 40.0)
EOBNRv2_22_q10_12236M.plot_pretty(t, hp, hc,2) 

t, hp, hc = EOBNRv2_44_q2_12238M(q = 1.7, M = 150.0, dist = 1.0, f_low=40.0)
EOBNRv2_44_q2_12238M.plot_pretty(t, hp, hc,3) 

plt.pyplot.show()